# CPI + CLI DATA COLLECTION


## OECD Composite Leading Indicator

In [ ]:
cli = obb.economy.composite_leading_indicator(start_date='2024-01-01', provider='oecd', country=['united_states'])
cli.to_polars()

## Consumer Price Index

### OECD Consumer Price Index

In [ ]:
oecd_cpi = obb.economy.cpi(start_date='2010-01-01', country='united_states', transform='yoy', provider='oecd',harmonized=False, expenditure='total')
oecd_cpi.results


### FRED Consumer Price Index

In [ ]:
fred_cpi = obb.economy.cpi(start_date='2024-07-01', country='united_states', transform='yoy', provider='fred')
fred_cpi.results

# humblCOMPASS 

In [30]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    # symbols=["AAPL"],
    # interval="1d",
    start_date="2023-01-01",
    end_date="2025-05-16",
    membership="humblPEON"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
compass = await toolbox.fundamental.humbl_compass(
    country="united_states",
    chart=True,
    recommendations=True
)



INFO: HumblCompassFetcher || START: fetch_data (async)


INFO: LogCacheHitPlugin || humbl_compass cache HIT & RETURNED
DEBUG: LogCacheHitPlugin || humbl_compass cache key: {"country": "united_states", "end_date": "2025-05-16", "start_date": "2023-05-17", "z_score": null}
INFO: HumblCompassFetcher || END: fetch_data (async) - Total time: 0.0183s


In [32]:
compass.to_polars()

shape: (24, 7)
┌──────────────────┬───────────────┬──────┬──────────────┬────────┬──────────────┬──────────────┐
│ date_month_start ┆ country       ┆ cpi  ┆ cpi_3m_delta ┆ cli    ┆ cli_3m_delta ┆ humbl_regime │
│ ---              ┆ ---           ┆ ---  ┆ ---          ┆ ---    ┆ ---          ┆ ---          │
│ date             ┆ str           ┆ f64  ┆ f64          ┆ f64    ┆ f64          ┆ str          │
╞══════════════════╪═══════════════╪══════╪══════════════╪════════╪══════════════╪══════════════╡
│ 2023-05-01       ┆ United States ┆ 4.05 ┆ -1.99        ┆ 98.71  ┆ 0.05         ┆ humblBOOM    │
│ 2023-06-01       ┆ United States ┆ 2.97 ┆ -2.02        ┆ 98.8   ┆ 0.16         ┆ humblBOOM    │
│ 2023-07-01       ┆ United States ┆ 3.18 ┆ -1.75        ┆ 98.89  ┆ 0.24         ┆ humblBOOM    │
│ 2023-08-01       ┆ United States ┆ 3.67 ┆ -0.38        ┆ 98.97  ┆ 0.26         ┆ humblBOOM    │
│ 2023-09-01       ┆ United States ┆ 3.7  ┆ 0.73         ┆ 99.03  ┆ 0.23         ┆ humblBOUNCE  │
│ …                ┆ …             ┆ …    ┆ …            ┆ …      ┆ …            ┆ …            │
│ 2024-12-01       ┆ United States ┆ 2.89 ┆ 0.45         ┆ 100.42 ┆ 0.46         ┆ humblBOUNCE  │
│ 2025-01-01       ┆ United States ┆ 3.0  ┆ 0.4          ┆ 100.5  ┆ 0.38         ┆ humblBOUNCE  │
│ 2025-02-01       ┆ United States ┆ 2.82 ┆ 0.07         ┆ 100.52 ┆ 0.23         ┆ humblBOUNCE  │
│ 2025-03-01       ┆ United States ┆ 2.39 ┆ -0.5         ┆ 100.49 ┆ 0.07         ┆ humblBOOM    │
│ 2025-04-01       ┆ United States ┆ 2.31 ┆ -0.69        ┆ 100.44 ┆ -0.06        ┆ humblBUST    │
└──────────────────┴───────────────┴──────┴──────────────┴────────┴──────────────┴──────────────┘

In [33]:
compass


HumblObject

id: 06827bb3-958b-7c05-8000-0d950e895bfb
results: b"DSL_VERSION\x01\x00\x00\x00\x81\xadDataFrameScan\x82\xa2df\xc5\x0cX\xff\...
equity_data: None
provider: yfinance
warnings: [{'category': 'HumblDataWarning', 'message': 'Start date adjusted to 2022...
chart: [{'content': '{"data":[{"hovertemplate":"\\u003cb\\u003e%{text}\\u003c\\u002...
extra: {'humbl_regime_recommendations': {'asset_classes': {'best': [<AssetRecommend...
context_params: {'symbols': 'AAPL', 'interval': '1d', 'start_date': datetime.date(2...
command_params: {'country': 'united_states', 'cli_start_date': '2023-01-17', 'cpi_s...
backtest: <function __autoreload_class__.humbl_compass.<locals>.backtest at 0x12d55...

In [34]:
compass.show()

In [19]:
compass.warnings

[Warning_(category='HumblDataWarning', message='Start date adjusted to 2022-12-02 based on humblPEON membership (2Y of data).'),
 Warning_(category='HumblDataWarning', message='Z-score defaulted to None. No z-score data will be calculated.')]

In [37]:
from aiocache import RedisCache


await RedisCache(namespace="humbl_compass").clear()

True

In [14]:
%load_ext autoreload
%autoreload 2

import polars as pl
from humbldata.core.standard_models.openbbapi.EconomyCompositeLeadingIndicatorQueryParams import EconomyCompositeLeadingIndicatorQueryParams
from humbldata.core.utils.openbb_api_client import OpenBBAPIClient


cli_query_params = EconomyCompositeLeadingIndicatorQueryParams(
    start_date="2022-08-02",
    end_date="2024-11-01",
    provider="oecd",
    country="united_states",
)
api_client = OpenBBAPIClient()
cli_response = await api_client.fetch_data(
    obb_path="economy.composite_leading_indicator",
    api_query_params=cli_query_params,
)
oecd_cli_data = cli_response.to_polars(collect=False)
oecd_cli_data = oecd_cli_data.rename({"value": "cli"}).with_columns(
    [pl.col("date").dt.month_start().alias("date_month_start")]
)

INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: http://127.0.0.1:6900/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2022-08-02&end_date=2024-11-01&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || Fetching data from: http://127.0.0.1:6900/api/v1/economy/composite_leading_indicator?provider=oecd&start_date=2022-08-02&end_date=2024-11-01&country=united_states&adjustment=amplitude&growth_rate=false
INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 0.1986s


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from humbldata.core.standard_models.openbbapi.EquityPriceHistoricalQueryParams import (
    EquityPriceHistoricalQueryParams,
)


equity_query_params = EquityPriceHistoricalQueryParams(
    symbol=["AAPL", "AMD"],
    start_date="2022-12-02",
    end_date="2024-11-01",
    provider="yfinance"
)

api_client = OpenBBAPIClient()
equity_response = await api_client.fetch_data(
    obb_path="equity.price.historical",
    api_query_params=equity_query_params,
)
equity_data = equity_response.to_polars(collect=False)


INFO: OpenBBAPIClient || START: fetch_data (async)
INFO: OpenBBAPIClient || Prepared request for: http://127.0.0.1:6900/api/v1/equity/price/historical?provider=yfinance&symbol=AAPL&symbol=AMD&start_date=2022-12-02&end_date=2024-11-01
INFO: OpenBBAPIClient || Fetching data from: http://127.0.0.1:6900/api/v1/equity/price/historical?provider=yfinance&symbol=AAPL&symbol=AMD&start_date=2022-12-02&end_date=2024-11-01


INFO: OpenBBAPIClient || END: fetch_data (async) - Total time: 0.2545s


In [27]:
equity_data.collect()

shape: (482, 6)
┌────────────┬────────────┬────────────┬────────────┬────────────┬──────────┐
│ date       ┆ open       ┆ high       ┆ low        ┆ close      ┆ volume   │
│ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---      │
│ date       ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ i64      │
╞════════════╪════════════╪════════════╪════════════╪════════════╪══════════╡
│ 2022-12-02 ┆ 75.050003  ┆ 76.279999  ┆ 73.650002  ┆ 74.980003  ┆ 64377200 │
│ 2022-12-05 ┆ 74.540001  ┆ 75.169998  ┆ 72.82      ┆ 73.620003  ┆ 47022200 │
│ 2022-12-06 ┆ 73.080002  ┆ 73.389999  ┆ 69.400002  ┆ 70.269997  ┆ 64965200 │
│ 2022-12-07 ┆ 69.529999  ┆ 71.370003  ┆ 69.279999  ┆ 70.139999  ┆ 59180100 │
│ 2022-12-08 ┆ 70.139999  ┆ 71.220001  ┆ 69.0       ┆ 70.470001  ┆ 56734100 │
│ …          ┆ …          ┆ …          ┆ …          ┆ …          ┆ …        │
│ 2024-10-28 ┆ 158.5      ┆ 160.279999 ┆ 157.039993 ┆ 159.919998 ┆ 36010100 │
│ 2024-10-29 ┆ 161.100006 ┆ 167.509995 ┆ 158.940002 ┆ 166.25     ┆ 71896000 │
│ 2024-10-30 ┆ 153.009995 ┆ 153.119995 ┆ 148.100006 ┆ 148.600006 ┆ 87701700 │
│ 2024-10-31 ┆ 147.800003 ┆ 148.679993 ┆ 143.330002 ┆ 144.070007 ┆ 44386600 │
│ 2024-11-01 ┆ 144.440002 ┆ 144.539993 ┆ 141.320007 ┆ 141.860001 ┆ 39027400 │
└────────────┴────────────┴────────────┴────────────┴────────────┴──────────┘